In [1]:
import pandas as pd

In [6]:
df_games = pd.read_parquet('src/cleaned/games.parquet')
df_genres = pd.read_parquet('src/cleaned/genres.parquet')
df_items = pd.read_parquet('src/cleaned/items.parquet')
df_reviews = pd.read_parquet('src/cleaned/reviews.parquet')
df_users = pd.read_parquet('src/cleaned/users.parquet')

In [8]:
df_items.loc[df_items['item_id'] == 10]

,item_id,item_name,playtime_forever,user_id
0,10,counter-strike,6,76561197970982479
277,10,counter-strike,0,js41637
1302,10,counter-strike,0,Riot-Punch
1635,10,counter-strike,93,doctr
3737,10,counter-strike,108,corrupted_soul
...,...,...,...,...
5150423,10,counter-strike,21,76561198273508956
5150611,10,counter-strike,1,76561198282090798
5150664,10,counter-strike,0,943525
5150692,10,counter-strike,6,76561198283312749
